In [4]:
%pylab inline
import pandas as pd
import os
DATA_DIR = '/Users/Will/Data/lending_club/'

Populating the interactive namespace from numpy and matplotlib


In [5]:
os.listdir(DATA_DIR)

['.DS_Store',
 'LoanStats3a.csv',
 'LoanStats3b.csv',
 'LoanStats3c.csv',
 'LoanStats3d.csv']

In [259]:
df = pd.DataFrame()
for csv in os.listdir(DATA_DIR)[1:]:
    df = df.append(pd.read_csv(DATA_DIR + csv, header=True))

/Users/Will/anaconda/envs/py34/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/Will/anaconda/envs/py34/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [261]:
important_columns = ['total_pymnt', 'zip_code', 'member_id', 'id', 'loan_amnt', 'int_rate',
                    'installment', 'emp_length', 'home_ownership', 'grade', 'sub_grade', 'emp_title',
                    'issue_d', 'loan_status', 'annual_inc', 'verification_status', 'purpose', 'addr_state', 'pymnt_plan',
                     'inq_last_6mths', 'dti', 'revol_util', 'mths_since_last_record', 'mths_since_last_delinq',
                     'pub_rec', 'revol_bal', 'open_acc', 'collections_12_mths_ex_med', 'delinq_2yrs',
                     'earliest_cr_line',  'last_credit_pull_d']

In [262]:
lc = df[important_columns]

In [128]:
other_columns = ['last_pymnt_d', 'last_pymnt_amnt']

In [263]:
mean(df.total_pymnt - df.total_pymnt_inv)

50.786928010183928

Earliest CR line is a good proxy for age.

# Type Casting

In [300]:
lc = lc[:-2] # The last 2 rows of the CSV arent proper rows

In [301]:
lc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 646387 entries, 0 to 180099
Data columns (total 30 columns):
total_pymnt                   646387 non-null float64
zip_code                      646387 non-null int64
member_id                     646387 non-null float64
id                            646387 non-null int64
loan_amnt                     646387 non-null float64
int_rate                      646387 non-null float64
installment                   646387 non-null float64
emp_length                    615969 non-null float64
home_ownership                646387 non-null category
grade                         646387 non-null category
sub_grade                     646387 non-null int64
emp_title                     609363 non-null object
issue_d                       646387 non-null datetime64[ns]
loan_status                   646387 non-null category
annual_inc                    646383 non-null float64
verification_status           646387 non-null int64
purpose                 

In [274]:
lc['zip_code'] = lc['zip_code'].str.replace('x', '')

In [275]:
lc.zip_code

0         860
1         309
2         606
3         917
4         972
5         852
6         280
7         900
8         958
9         774
10        853
11        913
12        245
13        606
14        951
15        641
16        921
17        067
18        890
19        921
20        770
21        335
22        799
23        917
24        900
25        605
26        103
27        150
28        326
29        564
         ... 
180072    925
180073    325
180074    209
180075    176
180076    339
180077    597
180078    412
180079    308
180080    840
180081    138
180082    282
180083    550
180084    871
180085    021
180086    330
180087    333
180088    891
180089    325
180090    286
180091    471
180092    956
180093    531
180094    430
180095    432
180096    791
180097    905
180098    072
180099    010
180100    378
180101    331
Name: zip_code, dtype: object

In [282]:
lc.zip_code = lc.zip_code.astype(int)

In [283]:
lc = lc[~lc.id.str.contains('[A-z]', na=False)] #Remove ids with words

In [284]:
lc['id'] = lc.id.astype(int)

In [285]:
lc.emp_length.unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', 'n/a'], dtype=object)

In [286]:
lc.emp_length = lc.emp_length.replace({'10+ years': 20, '< 1 year': 0, '1 year': 1})
lc.emp_length = lc.emp_length.replace({'{} years'.format(i): i for i in range(1,10)})
lc.emp_length = lc.emp_length.replace({'n/a': np.nan})

In [287]:
lc.int_rate = lc.int_rate.str.replace('%', '').astype(float) / 100

In [288]:
lc.home_ownership = lc.home_ownership.astype('category')

In [289]:
# Change the subgrades to numeric values

unique_subgrades = sorted(lc.sub_grade.unique())
lc.sub_grade = lc.sub_grade.replace({'{}'.format(i):
                      unique_subgrades.index(i) for i in unique_subgrades})

In [290]:
lc['grade'] = lc['grade'].astype('category')

In [291]:
lc.issue_d = pd.to_datetime(lc.issue_d)

In [292]:
lc.loan_status = lc.loan_status.astype('category')

In [293]:
lc.verification_status = lc.verification_status.replace({'VERIFIED - income': 1,
                                'VERIFIED - income source': 1, 'not verified': 0 })

In [294]:
lc.purpose = lc.purpose.astype('category')

In [295]:
lc.addr_state = lc.addr_state.astype('category')

In [296]:
lc = lc.drop('pymnt_plan', axis=1)

In [297]:
lc['revol_util'] = lc.revol_util.str.replace("%", '').astype(float) / 100

In [298]:
lc.earliest_cr_line = pd.to_datetime(lc.earliest_cr_line)

In [299]:
lc.last_credit_pull_d = pd.to_datetime(lc.last_credit_pull_d)

In [304]:
import pickle
with open('cleaned_df.pkl', 'wb') as picklefile:
    pickle.dump(lc, picklefile)

# Feature Engineering

Feature | Why
-- | --
Ratio of income to installment payment | might suggest overpayment amount
Ratio of income to all debt including installment | The previous but better.

In [310]:
lc[['installment', 'revol_bal']].mean()

installment      435.056483
revol_bal      16545.933747
dtype: float64

In [315]:
lc.columns

Index(['total_pymnt', 'zip_code', 'member_id', 'id', 'loan_amnt', 'int_rate',
       'installment', 'emp_length', 'home_ownership', 'grade', 'sub_grade',
       'emp_title', 'issue_d', 'loan_status', 'annual_inc',
       'verification_status', 'purpose', 'addr_state', 'inq_last_6mths', 'dti',
       'revol_util', 'mths_since_last_record', 'mths_since_last_delinq',
       'pub_rec', 'revol_bal', 'open_acc', 'collections_12_mths_ex_med',
       'delinq_2yrs', 'earliest_cr_line', 'last_credit_pull_d',
       'ratio_inc_debt'],
      dtype='object')

In [316]:
lc['ratio_inc_debt'] = (lc.loan_amnt + lc.revol_bal) / lc.annual_inc

In [317]:
lc.ratio_inc_debt.describe()

count    646383.000000
mean          0.458122
std           0.257535
min           0.001072
25%           0.277250
50%           0.418118
75%           0.592603
max          10.706455
Name: ratio_inc_debt, dtype: float64

In [322]:
lc['ratio_inc_installment'] = lc.installment / (lc.annual_inc / 12)

In [323]:
lc.ratio_inc_installment.describe()

count    646383.000000
mean          0.080259
std           0.040342
min           0.000241
25%           0.049286
50%           0.075108
75%           0.107037
max           0.541710
Name: ratio_inc_installment, dtype: float64

In [332]:
lc['ratio_mth_inc_all_payments'] = (lc.installment + lc.revol_bal * .02) / (lc.annual_inc / 12)

In [334]:
lc.ratio_mth_inc_all_payments.describe()

count    646383.000000
mean          0.137122
std           0.071193
min           0.000393
25%           0.085831
50%           0.126881
75%           0.177213
max           2.644211
Name: ratio_mth_inc_all_payments, dtype: float64

# Fixing some of the datatypes

Using 

In [335]:
lc.loan_status.value_counts()

Current                                                     428292
Fully Paid                                                  163412
Charged Off                                                  35389
Late (31-120 days)                                            9215
In Grace Period                                               5155
Does not meet the credit policy.  Status:Fully Paid           1959
Late (16-30 days)                                             1952
Does not meet the credit policy.  Status:Charged Off           761
Default                                                        223
Does not meet the credit policy.  Status:Current                28
Does not meet the credit policy.  Status:In Grace Period         1
dtype: int64

In [341]:
lc.loan_status = lc.loan_status.astype(str)

In [349]:
lc[lc.loan_status.str.contains("Does not meet the credit policy")].issue_d.max()

Timestamp('2010-12-23 00:00:00')

In [376]:
len(lc)

646387

In [375]:
df = df[~df.loan_status.str.contains("Does not meet the credit policy")]

,total_pymnt,zip_code,member_id,id,loan_amnt,int_rate,installment,emp_length,home_ownership,grade,...,pub_rec,revol_bal,open_acc,collections_12_mths_ex_med,delinq_2yrs,earliest_cr_line,last_credit_pull_d,ratio_inc_debt,ratio_inc_installment,ratio_mth_inc_all_payments
0,5861.071414,860,1296599,1077501,5000,0.1065,162.87,20,RENT,B,...,0,13648,3,0,0,1985-01-23,2015-01-23,0.777000,0.081435,0.217915
1,1008.710000,309,1314167,1077430,2500,0.1527,59.83,0,RENT,C,...,0,1687,3,0,0,1999-04-23,2013-09-23,0.139567,0.023932,0.037428
2,3003.653644,606,1313524,1077175,2400,0.1596,84.33,20,RENT,C,...,0,2956,2,0,0,2001-11-23,2015-09-23,0.437153,0.082595,0.140500
3,12226.302212,917,1277178,1076863,10000,0.1349,339.31,20,RENT,C,...,0,5598,10,0,0,1996-02-23,2015-01-23,0.317033,0.082759,0.110066
4,2971.010000,972,1311748,1075358,3000,0.1269,67.79,1,RENT,B,...,0,27783,15,0,0,1996-01-23,2015-09-23,0.384788,0.010169,0.093517
5,5631.377753,852,1311441,1075269,5000,0.0790,156.46,3,RENT,A,...,0,7963,9,0,0,2004-11-23,2015-09-23,0.360083,0.052153,0.105240
6,7456.520000,280,1304742,1069639,7000,0.1596,170.08,8,RENT,C,...,0,17726,7,0,0,2005-07-23,2015-09-23,0.526040,0.043421,0.133929
7,3938.144334,900,1288686,1072053,3000,0.1864,109.43,9,RENT,E,...,0,8221,4,0,0,2007-01-23,2014-12-23,0.233771,0.027358,0.068463
8,646.020000,958,1306957,1071795,5600,0.2128,152.39,4,OWN,F,...,0,5210,11,0,0,2004-04-23,2012-08-23,0.270250,0.045717,0.076977
9,1476.190000,774,1306721,1071570,5375,0.1269,121.45,0,RENT,B,...,0,9279,2,0,0,2004-09-23,2013-03-23,0.976933,0.097160,0.245624


In [384]:
import pandas as pd2

In [392]:
del lc['mths_since_last_record']

In [357]:
lc.issue_d.dt.year.value_counts()

2014    235629
2015    180100
2013    134756
2012     53367
2011     21721
2010     12537
2009      5281
2008      2393
2007       603
dtype: int64

In [336]:
1959 / (1959 + 761)

0.7202205882352941

In [337]:
163412 / (163412 + 35389)

0.8219878169626913

In [ ]:
lc.loan_status.str.replace('')

In [ ]:
lc.loan_status = lc.loan_status.rename({'Does not meet the credit policy.  Status:Fully Paid': 'Fully Paid',
                                       })

In [359]:
lc.loan_amnt.count()

646387

In [387]:
length = len(lc.loan_amnt)

In [366]:
lc.open_acc

0          3
1          3
2          2
3         10
4         15
5          9
6          7
7          4
8         11
9          2
10        14
11        12
12         4
13        11
14        14
15        11
16        14
17        20
18         4
19         8
20         8
21         7
22         8
23         6
24         7
25         8
26         7
27        14
28        12
29         8
          ..
180070     8
180071    13
180072    13
180073    15
180074     8
180075     8
180076     8
180077    13
180078     8
180079     9
180080    14
180081    10
180082     5
180083     9
180084    11
180085    11
180086     3
180087     9
180088     8
180089     9
180090     3
180091    13
180092    11
180093    10
180094     9
180095    13
180096     9
180097     9
180098     5
180099    17
Name: open_acc, dtype: float64

In [394]:
lc.loan_status.unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Late (31-120 days)',
       'In Grace Period', 'Late (16-30 days)', 'Default',
       'Does not meet the credit policy.  Status:Current',
       'Does not meet the credit policy.  Status:Charged Off',
       'Does not meet the credit policy.  Status:Fully Paid',
       'Does not meet the credit policy.  Status:In Grace Period'], dtype=object)

In [404]:
lc.issue_d.max()

Timestamp('2015-06-23 00:00:00')

In [401]:
lc['delinq']= 0
lc['delinq'][lc.loan_status.isin(['Charged Off', 'Late (31-120 days)', 'Default'])] = 1

/Users/Will/anaconda/envs/py34/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [403]:
lc['delinq'] = 0 if 9 else 1

# Exclude New Loans

We exclude newer loans because newer loans have not yet had much of a chance to become deliquent. 

In [412]:
sum(lc.issue_d < '2015-0-01')

585989

In [411]:
sum(lc.issue_d < '2014-09-01')

381537

In [414]:
df = df[df.issue_d < '2014-09-01']

Timestamp('2014-08-23 00:00:00')

# List of features

Feature | Description
-- | --
dti |A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
open_acc | Number of open accounts


In [370]:
(lc.dti - lc.ratio_inc_debt).mean()

-0.28082227667411802

In [369]:
lc.dti = lc.dti / 100